In [ ]:
#Invite everyone to the party
import pandas as pd
import numpy as np
import datetime as DT
import hmac
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
%matplotlib inline
from sqlalchemy import create_engine
import pyodbc 
from geopy import Point
from geopy.distance import distance



In [ ]:

#Connect to the database -- to be changed depending on configuration of the database

cnxn = pyodbc.connect(
    Trusted_Connection='Yes',
    Driver='{SQL Server Native Client 11.0}',
    Server='SERVER_NAME',
    Database='DATABASE_NAME'
)

cursor = cnxn.cursor()


In [ ]:
###Define dataframe for geocoded leads
customer_leads_g_query = """SELECT [Customer_id]
      ,[Full_name]
      ,[Customer_type]
      ,[Is_client_flg]
      ,[Is_referal]
      ,[Ref_Customer_id]
      ,[Country]
      ,[Region]
      ,[City]
      ,[Street]
      ,[Street_nr]
      ,[Activation_date]
      ,[Contact_phone]
      ,[Contact_email]
      ,[Acc_mgr_id]
      ,[Acc_mgr_name]
      ,[Full_address]
      ,[Lead_lat]
      ,[Lead_long]
      ,[Lead_address]
  FROM [DATABASE_NAME].[dbo].[Customer_leads_geocoded]"""



customer_leads_g = pd.read_sql(customer_leads_g_query,cnxn)

customer_leads_g.head()

In [ ]:
##Define dataframe for geocoded customers

customer_g_query = """SELECT [Customer_id]
      ,[Full_name]
      ,[Customer_type]
      ,[Is_client_flg]
      ,[Is_referal]
      ,[Ref_Customer_id]
      ,[Country]
      ,[Region]
      ,[City]
      ,[Street]
      ,[Street_nr]
      ,[Activation_date]
      ,[Contact_phone]
      ,[Contact_email]
      ,[Acc_mgr_id]
      ,[Acc_mgr_name]
      ,[full_address]
      ,[customer_lat]
      ,[customer_long]
      ,[customer_address]
  FROM [DATABASE_NAME].[dbo].[Customer_geocoded]
  where customer_type='Person'"""


customer_g = pd.read_sql(customer_g_query,cnxn)

customer_g.head()


In [ ]:
###Carthesian product of leads and customer 

customer_g['key'] = 0
customer_leads_g['key'] = 0

customer_lead_map = customer_g.merge(customer_leads_g, on='key', how='outer',suffixes=('_C','_L'))

customer_lead_map.head(5)

In [ ]:
##Cleanup the data
customer_lead_map = customer_lead_map[customer_lead_map['Lead_lat'].notna()]
customer_lead_map = customer_lead_map[customer_lead_map['customer_lat'].notna()]

In [ ]:
##Calculate_Distance between Customer and Lead
from geopy import Point
from geopy.distance import distance
customer_lead_map['s_point'] = customer_lead_map.apply(lambda row: Point(latitude=row['customer_lat'], 
                                                                           longitude=row['customer_long']), axis=1)

customer_lead_map['d_point'] = customer_lead_map.apply(lambda row: Point(latitude=row['Lead_lat'], 
                                                                           longitude=row['Lead_long']), axis=1)


customer_lead_map['distance_km'] = customer_lead_map.apply(lambda row: distance(row['s_point'], row['d_point']).km if row['d_point'] is not None else float('nan'), axis=1)



customer_lead_map.head()


In [ ]:

##Export Customer_Lead_mapping - and create it in database
customer_lead_map.to_csv (r'file/path', index = False) # place 'r' before the path name



In [ ]:
###SQL for tables

SQL_PIONEERS = """

select 
customer_id,
[Full_name_C],
[Region_C],
[City_C],
[Street_C],
[Street_nr_C],
[full_address],
[Contact_phone_C],
[Contact_email_C],
[Acc_mgr_id_C],
[Acc_mgr_name_C],
[customer_lat],
[customer_long],
[customer_address],
avg(cast (distance_km as float)) as Average_Distance,
count(distinct lead_id) as Nr_leads

into Customer_Pioneers

from [dbo].[Customer_Lead_relationship_Geocoded]
where cast (distance_km as float) < 1
--Filter for region
--and [Region_C]='Waikato'
group by 
customer_id,
[Full_name_C],
[Region_C],
[City_C],
[Street_C],
[Street_nr_C],
[full_address],
[Contact_phone_C],
[Contact_email_C],
[Acc_mgr_id_C],
[Acc_mgr_name_C],
[customer_lat],
[customer_long],
[customer_address]
having count(distinct lead_id) > 5

"""

In [ ]:
###Define pioneers dataframe from sql

pioneers_lead_map_query = """select * from Pioneers_Leads_MAP"""


pioneers_lead_map = pd.read_sql(pioneers_lead_map_query,cnxn)

pioneers_lead_map.head()

In [ ]:
SQL_PIONEERS_MAP = """select * into Pioneers_Leads_MAP
 from Customer_Pioneers_Lead_relationship
where [Customer_id] in (select Customer_id from [dbo].[Customer_Pioneers])
and cast (distance_km as float) < 1"""

In [ ]:
##Export Pioneer_Lead map and import into the table above
pioneers_lead_map.to_csv (r'file_path / Pioneer_Lead_MAP.csv', index = False) # place 'r' before the path name
